# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 24 2023 4:28PM,261715,10,MSP221080,"Methapharm, Inc.",Released
1,Apr 24 2023 4:28PM,261715,10,MSP221081,"Methapharm, Inc.",Released
2,Apr 24 2023 4:28PM,261715,10,MSP221082,"Methapharm, Inc.",Released
3,Apr 24 2023 4:28PM,261715,10,MSP221083,"Methapharm, Inc.",Released
4,Apr 24 2023 4:28PM,261715,10,MSP221085,"Methapharm, Inc.",Released
5,Apr 24 2023 4:28PM,261715,10,MSP221086,"Methapharm, Inc.",Released
6,Apr 24 2023 4:28PM,261715,10,MSP221087,"Methapharm, Inc.",Released
7,Apr 24 2023 4:28PM,261715,10,MSP221088,"Methapharm, Inc.",Released
8,Apr 24 2023 4:28PM,261715,10,MSP221089,"Methapharm, Inc.",Released
9,Apr 24 2023 4:23PM,261714,10,MSP221084-Hold_Rx(AL),"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,261711,Released,1
39,261712,Released,1
40,261713,Released,1
41,261714,Released,1
42,261715,Released,9


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261711,NaN,NaN,1.0
261712,NaN,NaN,1.0
261713,NaN,NaN,1.0
261714,NaN,NaN,1.0
261715,NaN,NaN,9.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261621,0.0,0.0,1.0
261622,0.0,0.0,50.0
261636,12.0,25.0,3.0
261641,0.0,0.0,1.0
261644,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261621,0,0,1
261622,0,0,50
261636,12,25,3
261641,0,0,1
261644,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261621,0,0,1
1,261622,0,0,50
2,261636,12,25,3
3,261641,0,0,1
4,261644,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261621,,,1
1,261622,,,50
2,261636,12,25,3
3,261641,,,1
4,261644,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 24 2023 4:28PM,261715,10,"Methapharm, Inc."
9,Apr 24 2023 4:23PM,261714,10,"Methapharm, Inc."
10,Apr 24 2023 4:23PM,261713,10,"Methapharm, Inc."
11,Apr 24 2023 4:22PM,261712,16,Sartorius Bioprocess Solutions
12,Apr 24 2023 4:19PM,261711,10,"Methapharm, Inc."
13,Apr 24 2023 4:14PM,261709,16,Sartorius Lab Products and Service
14,Apr 24 2023 4:10PM,261708,10,Methapharm-G
15,Apr 24 2023 4:07PM,261707,16,Sartorius Lab Products and Service
16,Apr 24 2023 4:00PM,261705,10,"ClearSpec, LLC"
17,Apr 24 2023 3:51PM,261701,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 24 2023 4:28PM,261715,10,"Methapharm, Inc.",,,9
1,Apr 24 2023 4:23PM,261714,10,"Methapharm, Inc.",,,1
2,Apr 24 2023 4:23PM,261713,10,"Methapharm, Inc.",,,1
3,Apr 24 2023 4:22PM,261712,16,Sartorius Bioprocess Solutions,,,1
4,Apr 24 2023 4:19PM,261711,10,"Methapharm, Inc.",,,1
5,Apr 24 2023 4:14PM,261709,16,Sartorius Lab Products and Service,,,1
6,Apr 24 2023 4:10PM,261708,10,Methapharm-G,,,1
7,Apr 24 2023 4:07PM,261707,16,Sartorius Lab Products and Service,,,1
8,Apr 24 2023 4:00PM,261705,10,"ClearSpec, LLC",,,1
9,Apr 24 2023 3:51PM,261701,19,"AdvaGen Pharma, Ltd",,,17


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 24 2023 4:28PM,261715,10,"Methapharm, Inc.",9,,
1,Apr 24 2023 4:23PM,261714,10,"Methapharm, Inc.",1,,
2,Apr 24 2023 4:23PM,261713,10,"Methapharm, Inc.",1,,
3,Apr 24 2023 4:22PM,261712,16,Sartorius Bioprocess Solutions,1,,
4,Apr 24 2023 4:19PM,261711,10,"Methapharm, Inc.",1,,
5,Apr 24 2023 4:14PM,261709,16,Sartorius Lab Products and Service,1,,
6,Apr 24 2023 4:10PM,261708,10,Methapharm-G,1,,
7,Apr 24 2023 4:07PM,261707,16,Sartorius Lab Products and Service,1,,
8,Apr 24 2023 4:00PM,261705,10,"ClearSpec, LLC",1,,
9,Apr 24 2023 3:51PM,261701,19,"AdvaGen Pharma, Ltd",17,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 24 2023 4:28PM,261715,10,"Methapharm, Inc.",9,,
1,Apr 24 2023 4:23PM,261714,10,"Methapharm, Inc.",1,,
2,Apr 24 2023 4:23PM,261713,10,"Methapharm, Inc.",1,,
3,Apr 24 2023 4:22PM,261712,16,Sartorius Bioprocess Solutions,1,,
4,Apr 24 2023 4:19PM,261711,10,"Methapharm, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 24 2023 4:28PM,261715,10,"Methapharm, Inc.",9.0,NaN,NaN
1,Apr 24 2023 4:23PM,261714,10,"Methapharm, Inc.",1.0,NaN,NaN
2,Apr 24 2023 4:23PM,261713,10,"Methapharm, Inc.",1.0,NaN,NaN
3,Apr 24 2023 4:22PM,261712,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
4,Apr 24 2023 4:19PM,261711,10,"Methapharm, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 24 2023 4:28PM,261715,10,"Methapharm, Inc.",9.0,0.0,0.0
1,Apr 24 2023 4:23PM,261714,10,"Methapharm, Inc.",1.0,0.0,0.0
2,Apr 24 2023 4:23PM,261713,10,"Methapharm, Inc.",1.0,0.0,0.0
3,Apr 24 2023 4:22PM,261712,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
4,Apr 24 2023 4:19PM,261711,10,"Methapharm, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2878660,19.0,15.0,2.0
15,784951,24.0,29.0,12.0
16,785128,3.0,0.0,0.0
18,1308295,5.0,0.0,0.0
19,785069,18.0,14.0,4.0
20,2354820,79.0,0.0,0.0
22,261664,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2878660,19.0,15.0,2.0
1,15,784951,24.0,29.0,12.0
2,16,785128,3.0,0.0,0.0
3,18,1308295,5.0,0.0,0.0
4,19,785069,18.0,14.0,4.0
5,20,2354820,79.0,0.0,0.0
6,22,261664,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,19.0,15.0,2.0
1,15,24.0,29.0,12.0
2,16,3.0,0.0,0.0
3,18,5.0,0.0,0.0
4,19,18.0,14.0,4.0
5,20,79.0,0.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,19.0
1,15,Released,24.0
2,16,Released,3.0
3,18,Released,5.0
4,19,Released,18.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,22
Status,,,,,,,
Completed,2.0,12.0,0.0,0.0,4.0,0.0,0.0
Executing,15.0,29.0,0.0,0.0,14.0,0.0,0.0
Released,19.0,24.0,3.0,5.0,18.0,79.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,22
0,Completed,2.0,12.0,0.0,0.0,4.0,0.0,0.0
1,Executing,15.0,29.0,0.0,0.0,14.0,0.0,0.0
2,Released,19.0,24.0,3.0,5.0,18.0,79.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,22
0,Completed,2.0,12.0,0.0,0.0,4.0,0.0,0.0
1,Executing,15.0,29.0,0.0,0.0,14.0,0.0,0.0
2,Released,19.0,24.0,3.0,5.0,18.0,79.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()